In [1]:
import os
from dotenv import load_dotenv
from transformers import AutoModelForSequenceClassification, AutoTokenizer, logging
import torch
import numpy as np

In [2]:
# Load tokenizer & model
logging.set_verbosity_error()
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny-mnli")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny")

/home/tony/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Load fine-tuned model weights
trained = torch.load('SST-2-BERT-tiny.bin', map_location=torch.device('cpu'))
model.load_state_dict(trained, strict=False)

/tmp/ipykernel_8956/2088438174.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained = torch.load('SST-2-BERT-tiny.bin', map_location=torch.device('cpu'))


_IncompatibleKeys(missing_keys=[], unexpected_keys=['bert.embeddings.position_ids'])

In [6]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [8]:
# Input data
text = "Nuovo Cinema Paradiso has been an incredible movie! A gem in the italian culture."
text = "[CLS] " + text + " [SEP]"

In [9]:
# Tokenize input data
#tokenized = tokenizer(text) # <- this would return the input_ids and attention mask as well
tokenized_text = tokenizer.tokenize(text) # <- tokenize the input data
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # <- convert tokens into ids
tokens_tensor = torch.tensor([indexed_tokens]) # <- convert token ids into tensor

#print("tokenized: ", tokenized)
#print("tokenized_text: ", tokenized_text)
#print("indexed_tokens: ", indexed_tokens)
#print("tokens_tensor: ", tokens_tensor)

# Embedding calculation
x = model.bert.embeddings(tokens_tensor, torch.tensor([[1] * len(tokenized_text)])) # require token ids tensor and attention mask list

print("x: ", x)

x:  tensor([[[ 0.7311, -0.0275, -8.5770,  ...,  1.3273,  0.1579, -0.1398],
         [ 0.0530, -0.2743, -0.2208,  ..., -1.4017, -0.5093, -0.7404],
         [-2.7270, -1.2554, -0.5465,  ..., -1.0224, -0.0679, -0.2995],
         ...,
         [-0.2928, -0.8412, -0.2279,  ..., -0.8374, -1.1446,  1.5150],
         [ 0.3317,  0.4604, -0.2388,  ...,  0.6477, -0.1716, -2.4884],
         [-1.4311,  0.4480, -0.1306,  ...,  1.0596, -1.3526, -0.9108]]],
       grad_fn=<NativeLayerNormBackward0>)


In [ ]:
# Save the embedding into text file
path = "./inputs/0"
for i in range(len(x[0])):
    if not (os.path.exists(path)):
        os.makedirs(path)
    np.savetxt("./inputs/0/input_{}.txt".format(i), x[0][i].detach(), delimiter=",")